0. Scan glass slides at 20x
1. Create Excel sheet with patient information
2. Screen H&E images to select potentially healthy sections
3. Annotate the Region of Interest (ROI) (Optional)
4. Convert ROI XML to Image (Code: roixml2png.py)
5. Convert H&E images to 1um resolution tif to apply DeepLab (Code: rescale_wsi.py)
6. Annotate 12 tissue compartments, apply DeepLab, and evaluate performance metrics
7. Apply HoVerNet
8. Rotate and crop tissue pieces and DLmask (Code)
9. Rotate HoVerNet segmented nuclei mask (Code)
10. Calculate tissue composition (Code)
11. Calculate epidermis thickness, waviness, roughness
12. Calculate dermis thickness, waviness
13. Calculate ECM anisotropy,Interstitial orientation, Thick gap ratio, Thick gap average
14. Calculate tissue morphology
15. ECM cell count and morphology + std using Manual Threshold method
16. Distance from ECM cell to tissue
17. All celltype morphology, orientation, and distance to tissue using HoVerNet
18. Further add features such as more detailed cell subtyping

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from natsort import natsorted

#utilities
from roixml2png import roixml2png
from rescale_wsi import rescale_wsi

In [2]:
NDPI = r'\\fatherserverdw\kyuex\clue images'
oneum = r'\\fatherserverdw\kyuex\clue images\1um'
imcrop = r'\\fatherserverdw\kyuex\clue images\1um\imcrop'
imcroproi = r'\\fatherserverdw\kyuex\clue images\1um\imcrop_roi'

dlmask = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dlcrop = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop'
dlcroproi = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop_roi'

roixml = r'\\fatherserverdw\kyuex\clue images\annotations\roi'
roimask_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
tissue_area_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\TA_20rsf'

twelve_annot = r'\\fatherserverdw\kyuex\clue images\annotations\12class'

LUT = r"\\fatherserverdw\kyuex\imlist_all.xlsx"

In [3]:
#Load healthy tissue look up table (LUT)
LUT = pd.read_excel(LUT)
# xmlist = LUT['filename'][(LUT['student score']>1) & (LUT['Block or Slide?']=="Both")]
LUThealthy = LUT[(LUT['student score']>1)]

In [5]:
natsorted(LUThealthy.filename)

['2022-06-07 13.18.40.ndpi',
 '2022-06-07 13.29.45.ndpi',
 '2022-06-07 13.51.05.ndpi',
 '2022-06-07 14.27.44.ndpi',
 '2022-06-07 14.32.24.ndpi',
 '2022-06-07 14.34.59.ndpi',
 '2022-06-07 14.37.19.ndpi',
 '2022-06-07 14.39.32.ndpi',
 '2022-06-07 14.56.13.ndpi',
 '2022-06-07 16.05.45.ndpi',
 '2022-06-07 16.10.49.ndpi',
 '2022-06-07 17.05.33.ndpi',
 '2022-06-07 17.10.29.ndpi',
 '2022-06-07 17.15.33.ndpi',
 '2022-06-07 17.24.15.ndpi',
 '2022-06-07 17.43.53.ndpi',
 '2022-06-07 17.48.30.ndpi',
 '2022-06-07 18.06.50.ndpi',
 '2022-06-08 16.05.29.ndpi',
 '2022-06-08 16.34.23.ndpi',
 '2022-06-08 16.52.37.ndpi',
 '2022-06-08 17.04.20.ndpi',
 '2022-06-08 17.14.34.ndpi',
 '2022-06-08 17.25.47.ndpi',
 '2022-06-08 17.33.23.ndpi',
 '2022-06-08 17.38.55.ndpi',
 '2022-06-08 17.54.57.ndpi',
 '2022-06-08 18.08.15.ndpi',
 '2022-06-08 18.13.05.ndpi',
 '2022-06-08 18.23.12.ndpi',
 '2022-06-08 18.27.20.ndpi',
 '2022-06-09 11.32.55.ndpi',
 '2022-06-09 11.36.47.ndpi',
 '2022-06-09 11.46.15.ndpi',
 '2022-06-09 1

In [10]:
#Step 4: ROI XML 2 Image
#Saves Image to where XML is in subfolders
LUThealthyback = LUThealthy[LUThealthy['body part 1'].str.lower()=="back"]
xmlist = [os.path.join(roixml,xmlpth.replace('ndpi','xml')) for xmlpth in LUThealthyback['filename']]
roiinfo = [roixml2png(xmlpth,NDPI) for xmlpth in xmlist]
# pd.DataFrame(np.array(roiinfo),columns=['fn','ROIA','TA','ratio']).to_csv(r"\\fatherserverdw\kyuex\ROITA_ratio.csv")

2022-06-14 17.39.31.xml
2022-06-15 16.15.49.xml
2022-06-15 16.20.00.xml
2022-06-07 17.24.15.xml


C:\Users\NDP\AppData\Local\Temp\ipykernel_6696\128553994.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pd.DataFrame(np.array(roiinfo),columns=['fn','ROIA','TA','ratio']).to_csv(r"\\fatherserverdw\kyuex\ROITA_ratio.csv")


ValueError: Shape of passed values is (66, 1), indices imply (66, 4)

In [11]:
roiinfo

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['2022-06-14 17.39.31.xml', 478181, 1506182, 32],
 None,
 None,
 None,
 ['2022-06-15 16.15.49.xml', 781666, 3044734, 26],
 ['2022-06-15 16.20.00.xml', 669730, 3474527, 19],
 None,
 ['2022-06-07 17.24.15.xml', 469388, 1986936, 24]]

In [ ]:
#Step 5: rescale wsi to 1um for applying deeplab
#Saves Image to where wsi is in subfolder
wsis = [os.path.join(NDPI,_) for _ in LUThealthy.filename]
wsis = natsorted(wsis)
for idx,wsi in enumerate(wsis):
    rescale_wsi(wsi,1)
    print(idx,'/',len(wsis))